In [1]:
import csv
import random
import pandas as pd
import numpy as np
import os
import seaborn as sns
from time import time
import matplotlib.pyplot as plt
import scipy.io
import pandas as pd
import folium
from folium.plugins import AntPath
import plotly.express as px
from datetime import datetime
import glob

In [ ]:
#pip install pandas matplotlib seaborn folium plotly


In [26]:
df = pd.read_csv('output_file.csv')

In [3]:
df

,GPS-Validity,latitude,longitude,speed,engine-status,label,timestamp
0,1,35.755108,51.549366,27,1,0,2024-06-02 11:06:51
1,1,35.755108,51.549366,0,1,0,2024-06-02 11:07:50
2,1,35.755108,51.549366,0,0,0,2024-06-02 11:07:51
3,1,35.755108,51.549366,0,1,0,2024-06-02 11:13:24
4,1,35.755108,51.549366,0,1,0,2024-06-02 11:13:25
...,...,...,...,...,...,...,...
248,1,35.758846,51.550503,0,0,0,2024-06-02 17:49:48
249,1,35.758751,51.550587,0,0,0,2024-06-02 18:04:59
250,1,35.758774,51.550667,0,0,0,2024-06-02 18:19:59
251,1,35.758652,51.550678,0,0,0,2024-06-02 18:34:59


# Plotting the Route on a Map

In [8]:
center_lat = df['latitude'].mean()
center_lon = df['longitude'].mean()
m = folium.Map(location=[center_lat, center_lon], zoom_start=13)
route = list(zip(df['latitude'], df['longitude']))
folium.PolyLine(route, color='blue', weight=5, opacity=0.8).add_to(m)
m.save('route_map.html')

# Interactive Route with Speed Popups
Add popups to the map that display speed when markers are clicked.

# Show Spoof Labels in the Map

In [22]:
from folium.plugins import MarkerCluster
import folium

center_lat = df['latitude'].mean()
center_lon = df['longitude'].mean()

m = folium.Map(location=[center_lat, center_lon], zoom_start=13)

marker_cluster = MarkerCluster().add_to(m)

for idx, row in df.iterrows():
    # Customize the popup to include speed and the label (spoof or not)
    popup_text = f"Speed: {row['speed']} km/h<br>Label: {row['label']}"
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=popup_text
    ).add_to(marker_cluster)

m.save('clustered_route_with_labels_map.html')


# Mapping Points with Clustering

In [16]:
from folium.plugins import MarkerCluster

m = folium.Map(location=[center_lat, center_lon], zoom_start=13)

marker_cluster = MarkerCluster().add_to(m)

for idx, row in df.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=f"Speed: {row['speed']} km/h"
    ).add_to(marker_cluster)

m.save('clustered_route_map.html')

# Interactive Map with Clickable Markers

In [20]:
import folium
from folium.plugins import MarkerCluster

# Define the function to log clicked data in JavaScript
click_js = """
function logData(lat, lon, speed, label) {
    console.log('Latitude: ' + lat + ', Longitude: ' + lon + ', Speed: ' + speed + ' km/h, Label: ' + label);
}
"""

center_lat = df['latitude'].mean()
center_lon = df['longitude'].mean()
m = folium.Map(location=[center_lat, center_lon], zoom_start=13)

# Create a marker cluster
marker_cluster = MarkerCluster().add_to(m)

# Add markers with click events
for idx, row in df.iterrows():
    lat = row['latitude']
    lon = row['longitude']
    speed = row['speed']
    label = row['label'] if 'label' in row else 'Unknown'  # Default if label not available

    # Create a marker with a popup containing lat, lon, speed, and label
    folium.Marker(
        location=[lat, lon],
        popup=f"Speed: {speed} km/h<br>Label: {label}",
        # Use JavaScript to log the data of the clicked marker
        onclick=f"logData({lat}, {lon}, {speed}, '{label}')"
    ).add_to(marker_cluster)

m.get_root().script.add_child(folium.Element(click_js))

# Save the map
m.save('interactive_map_with_clicks.html')


In [18]:
import folium
from folium.plugins import MarkerCluster
import ipywidgets as widgets
from IPython.display import display
import json

center_lat = df['latitude'].mean()
center_lon = df['longitude'].mean()
m = folium.Map(location=[center_lat, center_lon], zoom_start=13)

marker_cluster = MarkerCluster().add_to(m)

clicked_data = []


output = widgets.Output()
save_button = widgets.Button(description="Save Data")


def save_data(_):
    with open('clicked_data.json', 'w') as f:
        json.dump(clicked_data, f)
    with output:
        output.clear_output()
        print("Data saved to clicked_data.json")


save_button.on_click(save_data)


def log_click(lat, lon, speed, label):
    data = {
        "Latitude": lat,
        "Longitude": lon,
        "Speed": speed,
        "Label": label
    }
    clicked_data.append(data)
    with output:
        output.clear_output()
        print(f"Clicked Data: {data}")


for idx, row in df.iterrows():
    lat = row['latitude']
    lon = row['longitude']
    speed = row['speed']
    label = row['label'] if 'label' in row else 'Unknown'


    folium.Marker(
        location=[lat, lon],
        popup=f"Speed: {speed} km/h<br>Label: {label}",
        onclick=f"log_click({lat}, {lon}, {speed}, '{label}')"
    ).add_to(marker_cluster)

display(m)

display(save_button, output)


Button(description='Save Data', style=ButtonStyle())

Output()

In [19]:
import json
with open('clicked_data.json', 'r') as f:
    data = json.load(f)
    print(data)

[]


In [21]:
def log_click(lat, lon, speed, label):
    data = {
        "Latitude": lat,
        "Longitude": lon,
        "Speed": speed,
        "Label": label
    }
    clicked_data.append(data)
    print(f"Collected Data: {data}")  
    with output:
        output.clear_output()
        print(f"Clicked Data: {data}")


In [22]:
def save_data(_):
    if clicked_data:  
        with open('clicked_data.json', 'w') as f:
            json.dump(clicked_data, f)
        with output:
            output.clear_output()
            print("Data saved to clicked_data.json")
            print(f"Saved Data: {clicked_data}")  # Debugging step
    else:
        print("No data to save!")

In [24]:
pip install branca

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


# Enabling Area Selection

In [25]:
import folium
from folium.plugins import MarkerCluster

center_lat = df['latitude'].mean()
center_lon = df['longitude'].mean()
m = folium.Map(location=[center_lat, center_lon], zoom_start=13)

# Create a marker cluster
marker_cluster = MarkerCluster().add_to(m)

# Define a color scheme based on label
def get_marker_color(label):
    if label == 'spoof':
        return 'red'  # Spoofed points will be red
    else:
        return 'green'  # Non-spoofed points will be green

for idx, row in df.iterrows():
    lat = row['latitude']
    lon = row['longitude']
    speed = row['speed']
    label = row['label'] if 'label' in row else 'Unknown'  # Default if label not available

    marker_color = get_marker_color(label)

    folium.CircleMarker(
        location=[lat, lon],
        radius=5,
        color=marker_color,
        fill=True,
        fill_color=marker_color,
        fill_opacity=0.7,
        popup=f"Speed: {speed} km/h<br>Label: {label}"
    ).add_to(marker_cluster)

draw = folium.plugins.Draw(export=True)
draw.add_to(m)

js_function = """
<script>
function captureSelectedArea(e) {
    var layer = e.layer;
    var type = e.layerType;

    if (type === 'rectangle' || type === 'polygon') {
        var bounds = layer.getBounds();
        console.log('Bounds:', bounds);

        // Here you can add custom code to filter points within the bounds
        // For example, log points within bounds
    }
}

map.on('draw:created', captureSelectedArea);
</script>
"""
m.get_root().html.add_child(folium.Element(js_function))

m.save('selectable_map_with_bulk_selection.html')


# Steps to Set Up Backend Integration for Saving Selected Points

In [26]:
pip install flask


   ---------------------------------------- 0.0/101.7 kB ? eta -:--:--
   ---------------------------------------- 0.0/101.7 kB ? eta -:--:--
   ---- ----------------------------------- 10.2/101.7 kB ? eta -:--:--
   ----------- --------------------------- 30.7/101.7 kB 445.2 kB/s eta 0:00:01
   ----------- --------------------------- 30.7/101.7 kB 445.2 kB/s eta 0:00:01
   ----------------------- --------------- 61.4/101.7 kB 365.7 kB/s eta 0:00:01
   ----------------------------------- --- 92.2/101.7 kB 403.5 kB/s eta 0:00:01
   -------------------------------------- 101.7/101.7 kB 419.7 kB/s eta 0:00:00
   ---------------------------------------- 0.0/97.9 kB ? eta -:--:--
   ---------------- ----------------------- 41.0/97.9 kB 2.0 MB/s eta 0:00:01
   ------------------------------------- -- 92.2/97.9 kB 1.7 MB/s eta 0:00:01
   ---------------------------------------- 97.9/97.9 kB 1.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/227.6 kB ? eta -:--:--
   -------


[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


### simple Flask app that listens for requests and saves selected points to a CSV file.

attention to file app.py

# Modify JavaScript in the Folium Map
modify the JavaScript in folium map to send the selected points to the Flask server when we draw an area on the map.

In [24]:
import folium
from folium.plugins import MarkerCluster

center_lat = df['latitude'].mean()
center_lon = df['longitude'].mean()
m = folium.Map(location=[center_lat, center_lon], zoom_start=13)


marker_cluster = MarkerCluster().add_to(m)

def get_marker_color(label):
    if label == 'spoof':
        return 'red'  # Spoofed points will be red
    else:
        return 'green'  # Non-spoofed points will be green


for idx, row in df.iterrows():
    lat = row['latitude']
    lon = row['longitude']
    speed = row['speed']
    label = row['label'] if 'label' in row else 'Unknown'  

   
    marker_color = get_marker_color(label)

  
    folium.CircleMarker(
        location=[lat, lon],
        radius=5,
        color=marker_color,
        fill=True,
        fill_color=marker_color,
        fill_opacity=0.7,
        popup=f"Speed: {speed} km/h<br>Label: {label}"
    ).add_to(marker_cluster)

draw = folium.plugins.Draw(export=True)
draw.add_to(m)

js_function = """
<script>
function captureSelectedArea(e) {
    var layer = e.layer;
    var type = e.layerType;

    if (type === 'rectangle' || type === 'polygon') {
        var bounds = layer.getBounds();
        
        // Get points inside the selected bounds
        var selectedPoints = [];
        marker_cluster.eachLayer(function(layer) {
            var lat = layer.getLatLng().lat;
            var lon = layer.getLatLng().lng;
            
            // Check if point is inside the selected bounds
            if (bounds.contains([lat, lon])) {
                selectedPoints.push({
                    lat: lat,
                    lon: lon,
                    speed: layer.getPopup().getContent().match(/Speed: (\\d+\\.?\\d*)/)[1], // Extract speed from popup
                    label: layer.getPopup().getContent().match(/Label: (.+)/)[1]  // Extract label from popup
                });
            }
        });
        
        // Send the selected points to the Flask backend
        fetch('http://127.0.0.1:5000/save_points', {
            method: 'POST',
            headers: {
                'Content-Type': 'application/json'
            },
            body: JSON.stringify({points: selectedPoints})
        })
        .then(response => response.json())
        .then(data => {
            console.log('Response:', data);
            alert(data.message);
        })
        .catch(error => {
            console.error('Error:', error);
            alert('Failed to save points.');
        });
    }
}

map.on('draw:created', captureSelectedArea);
</script>
"""
m.get_root().html.add_child(folium.Element(js_function))

m.save('selectable_map_with_flask.html')


## Run the Flask server:
python flask_app.py